# Cosinor Regression Model used for 24-hours Window Analysis

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.ma import add
from CosinorPy import file_parser, cosinor, cosinor1
np.seterr(divide='ignore')
import scipy.signal as signal
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.multitest as multi
from scipy.optimize import curve_fit
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from scipy.stats import percentileofscore
from scipy.stats import circstd, circmean

import copy
import itertools
from random import sample
import os
import copy
from CosinorPy.helpers import df_add_row

### Data Preparation

In [6]:
def load_data(path):
    """Function used to parser the data using the file_parser method from the CosinorPy package"""
    df = pd.read_csv(path)
    df['Region start time'] = pd.to_datetime(df['Region start time'])
    df['date'] = df['Region start time'].dt.date
    df['hour'] = df['Region start time'].dt.hour
    df = df.drop('Unnamed: 0', axis=1)
    return df

In [7]:
pre_data = load_data("data/RNS_G_Pre_output.csv")
post_data = load_data("data/RNS_G_M1_output.csv")

In [ ]:
pre_data_24h = pre_data.groupby(['date', 'hour'])
cosinor.plot_data(pre_data)

,Region start time,Pattern A Channel 2,Episode starts with RX,date,hour
0,2024-11-19 10:00:00,0,0,2024-11-19,10
1,2024-11-19 11:00:00,19,0,2024-11-19,11
2,2024-11-19 12:00:00,38,0,2024-11-19,12
3,2024-11-19 13:00:00,1,0,2024-11-19,13
4,2024-11-19 14:00:00,3,0,2024-11-19,14
...,...,...,...,...,...
705,2024-12-18 19:00:00,40,0,2024-12-18,19
706,2024-12-18 20:00:00,68,0,2024-12-18,20
707,2024-12-18 21:00:00,87,0,2024-12-18,21
708,2024-12-18 22:00:00,77,0,2024-12-18,22
